# P7 (Соціальне забезпечення та соціальний захист населення): 5/5

In [1]:
import pandas as pd
from src.utils import normalize_parameter, save_data
from src.utils import PATH_INTERIM

## p7_01
### кількість витрачених коштів на придбання технічних та інших засобів реабілітації у розрахунку на одну особу з інвалідністю

In [2]:
df_P07_002 = pd.read_excel(PATH_INTERIM / "P7" / "P07_002.xlsx")
df_P07_002["p7_01_raw"] = (
    df_P07_002["Кількість технічних засобів реабілітації"] / 
    df_P07_002["Кількість осіб"]
)
df_P07_002["p7_01"] = normalize_parameter(df_P07_002["p7_01_raw"])

feature_range=(0, 1); fill_na=True; array_bounds=(0.0, 0.024266365688487584); normalization_bounds=(0.0, 0.024266365688487584), reverse=False


## p7_02
### частка дітей, що перебуває на обліку в інклюзивно-ресурсних центрах
## p7_06
### частка дітей, що здобуває освіту в інклюзивних групах ЗДО та інклюзивних класах ЗЗСО

In [3]:
relevant_columns = [
    "Кількість дітей на території обслуговування ІРЦ",
    "Кількість дітей, які перебувають на обліку в ІРЦ",
    "Кількість дітей у віці 2-5 років, які здобувають освіту в інклюзивних групах ЗДО",
    "Кількість дітей у віці 6-18 років, які здобувають освіту в інклюзивних класах ЗЗСО"
]

df_P07_004 = pd.read_excel(PATH_INTERIM / "P7" / "P07_004.xlsx")
df_P07_004_fin = df_P07_004.groupby("region", as_index=False)[relevant_columns].sum()
df_P07_004_fin["Здобувають освіту в інкл. класах ЗДО/ЗЗСО"] = (
    df_P07_004_fin["Кількість дітей у віці 2-5 років, які здобувають освіту в інклюзивних групах ЗДО"] +
    df_P07_004_fin["Кількість дітей у віці 6-18 років, які здобувають освіту в інклюзивних класах ЗЗСО"]
)

df_P07_004_fin["p7_02_raw"] = (
    df_P07_004_fin["Кількість дітей, які перебувають на обліку в ІРЦ"] / 
    df_P07_004_fin["Кількість дітей на території обслуговування ІРЦ"]
)

df_P07_004_fin["p7_06_raw"] = (
    df_P07_004_fin["Здобувають освіту в інкл. класах ЗДО/ЗЗСО"] / 
    df_P07_004_fin["Кількість дітей на території обслуговування ІРЦ"]
)

df_P07_004_fin["p7_02"] = normalize_parameter(df_P07_004_fin["p7_02_raw"], min_bound=0)
df_P07_004_fin["p7_06"] = normalize_parameter(df_P07_004_fin["p7_06_raw"], min_bound=0, max_bound=1)

feature_range=(0, 1); fill_na=True; array_bounds=(0.003812154271074558, 0.036411706837186426); normalization_bounds=(0, 0.036411706837186426), reverse=False
feature_range=(0, 1); fill_na=True; array_bounds=(0.0011635527778041622, 0.009047495537280868); normalization_bounds=(0, 1), reverse=False


## p7_03
### частка дітей-сиріт та дітей, позбавлених батьківського піклування, яких влаштували до сімейних форм виховання або знаходяться під опікою, піклуванням
## p7_07
### кількість наданих соціальних послуг центрами обліку бездомних громадян

In [4]:
df_P07_005 = pd.read_excel(PATH_INTERIM / "P7" / "P07_005.xlsx")
df_P07_005["p7_07_raw"] = df_P07_005["Надано соціальні послуги, осіб"]
df_P07_005["p7_07"] = normalize_parameter(df_P07_005["p7_07_raw"])

feature_range=(0, 1); fill_na=True; array_bounds=(0, 3115); normalization_bounds=(0, 3115), reverse=False


In [5]:
df_P07_006 = pd.read_excel(PATH_INTERIM / "P7" / "P07_006.xlsx").rename(columns={"Регіон": "region"})

df_P07_006["p7_03_raw"] = (
    (df_P07_006["Кількість дітей-сиріт і дітей, позбавлених батьківського піклування, які знаходяться у прийомних сім’ях, дитячих будинках сімейного типу"]+df_P07_006['Кількість дітей-сиріт і дітей, позбавлених батьківського піклування, які знаходяться під опікою, піклуванням ']) / 
    df_P07_006["Загальна кількість дітей-сиріт і дітей, позбавлених батьківського піклування станом на 1.10.2020"]
)
df_P07_006["p7_03"] = normalize_parameter(df_P07_006["p7_03_raw"], min_bound=0, max_bound=1)

feature_range=(0, 1); fill_na=True; array_bounds=(0.0, 1.012393848978655); normalization_bounds=(0, 1), reverse=False


## Група 1: p7_01, люди з інвалідністю
## Група 2: p7_02-p7_03/p7_07, діти з особливими освітніми потребами
## Група 3: p7_06, бездомні

In [6]:
dict_weights = {
    'p7_01':1,
    'p7_02':1,
    'p7_03':1,
    'p7_06':1,
    'p7_07':1
}

sources = [df_P07_002, df_P07_004_fin, df_P07_005, df_P07_006]
save_data(sources, dict_weights, "P7", show_results=True)

,region,p7_01,p7_02,p7_06,p7_07,p7_03,P7
0,Вінницька,0.328904,0.437409,0.004341,0.118780,0.987397,3.753661
1,Волинська,0.067247,0.418038,0.004744,0.003852,0.949967,2.887695
2,Дніпропетровська,0.120130,0.123518,0.001164,0.816693,0.905594,3.934198
3,Донецька,0.122811,1.000000,0.004902,0.845265,0.897035,5.740027
4,Житомирська,0.008852,0.448698,0.003964,0.016372,0.955882,2.867537
5,Закарпатська,0.012029,0.311199,0.004248,0.154093,0.861286,2.685710
6,Запорізька,0.435616,0.784071,0.005515,0.171108,0.910013,4.612646
7,Івано-Франківська,0.067085,0.338067,0.005134,0.074799,0.880372,2.730915
8,Київська,1.000000,NaN,NaN,0.019583,0.922730,NaN
9,Кіровоградська,0.095861,0.515756,0.008176,0.143820,0.932395,3.392018
